<a href="https://colab.research.google.com/github/HARRYPLONES1/PORTAFILIO_IA/blob/main/Cuanti_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import BertModel, BertTokenizer

# Cargar el modelo BERT preentrenado
model_name = "bert-base-uncased"
model = BertModel.from_pretrained(model_name)

# Cargar el tokenizador de BERT
tokenizer = BertTokenizer.from_pretrained(model_name)

# Crear un ejemplo de texto para la inferencia
text = "This is a sample sentence to be processed by BERT."
inputs = tokenizer(text, return_tensors="pt")

# Verificar el modelo original
print(model)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from torch.quantization import quantize_dynamic
import torch.nn as nn

# Aplicar la cuantización dinámica solo a las capas nn.Linear del transformer
quantized_model = quantize_dynamic(
    model,  # El modelo transformer
    {nn.Linear},  # Tipos de capas a cuantizar
    dtype=torch.qint8  # Usar enteros de 8 bits (INT8) para la cuantización
)

# Verificar el modelo cuantizado
print(quantized_model)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (key): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (value): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): DynamicQuant

In [ ]:
import os

# Guardar el modelo original y cuantizado
torch.save(model.state_dict(), "original_bert.pth")
torch.save(quantized_model.state_dict(), "quantized_bert.pth")

# Comparar el tamaño de los modelos
original_size = os.path.getsize("original_bert.pth")
quantized_size = os.path.getsize("quantized_bert.pth")

print(f"Tamaño del modelo original: {original_size / 1e6:.2f} MB")
print(f"Tamaño del modelo cuantizado: {quantized_size / 1e6:.2f} MB")


Tamaño del modelo original: 438.01 MB
Tamaño del modelo cuantizado: 181.49 MB


In [ ]:
import time

# Tiempo de inferencia con el modelo original
start = time.time()
for _ in range(100):
    with torch.no_grad():
        model(**inputs)
end = time.time()
print(f"Tiempo de inferencia del modelo original: {end - start:.4f} segundos")

# Tiempo de inferencia con el modelo cuantizado
start = time.time()
for _ in range(100):
    with torch.no_grad():
        quantized_model(**inputs)
end = time.time()
print(f"Tiempo de inferencia del modelo cuantizado: {end - start:.4f} segundos")


Tiempo de inferencia del modelo original: 10.6936 segundos
Tiempo de inferencia del modelo cuantizado: 6.3033 segundos
